# Decorators
- article: https://realpython.com/primer-on-python-decorators

In [53]:
import statistics
from functools import lru_cache, wraps

## Example: @lru_cache

In [4]:
def fibo(n):
    match n:
        case 0 | 1:
            return n
        case _:
            return fibo(n-1) + fibo(n-2)

In [5]:
for i in range(10):
    print(fibo(i))

0
1
1
2
3
5
8
13
21
34


In [6]:
fibo(40)

102334155

In [7]:
def fibo_cache(n):
    @lru_cache
    # @lru_cache(maxsize=1024)
    def fibo_internal(n):
        match n:
            case 0 | 1:
                return n
            case _:
                return fibo_internal(n-1) + fibo_internal(n-2)
    return fibo_internal(n)

In [8]:
fibo_cache(40)

102334155

In [9]:
fibo_cache(50)

12586269025

In [10]:
# 7 x batch of 1000 calls
%timeit -n 1000 fibo_cache(40)

17.2 μs ± 3.11 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
# 2 x batch of 1 call
%timeit -n 1 -r 2 fibo(40)

24.2 s ± 844 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [77]:
def calcul(x):
    "doc of calcul function"
    return x**2 + 1

# a decorator is a function !
# calcul2 = lru_cache(calcul)
calcul2 = lru_cache(maxsize=1024)(calcul)

In [79]:
print(calcul2(3))
print(calcul2(3))
print(calcul2(3))
print(calcul2(2))

10
10
10
5


In [ ]:
calcul2?

In [ ]:
calcul2.__name__

## Decorator @logger

In [17]:
def logger(f):
    @wraps(f)
    def logger_wrapper(*args, **kwargs):
        print(f'** function called with positional args: {args} and keyword args: {kwargs}')
        res = f(*args, **kwargs)
        print(f'** return value: {res}')
        return res
    return logger_wrapper

In [39]:
@logger
def generate():
    print('generate ....')

In [41]:
generate()

** function called with positional args: () and keyword args: {}
generate ....
** return value: None


In [43]:
@logger
def calcul(x):
    return x**2 + 1

In [ ]:
y = calcul(3)
y

In [ ]:
y = calcul(x=3)
y

In [ ]:
zip = logger(zip)
for a,b,c in zip('abcd', (1,2,10), range(20)):
    print(a,b,c)

## decorator @twice
- execute twice decorated function
- return last result

In [19]:
def twice(f):
    @wraps(f)
    def twice_wrapper(*args, **kwargs):
        f(*args, **kwargs)
        return f(*args, **kwargs)
    # twice_wrapper.__doc__ = f.__doc__
    # twice_wrapper.__name__ = f.__name__
    return twice_wrapper

In [21]:
@twice
@logger
def compute(x, y):
    return (x**2 + y**2)**.5

In [23]:
compute(3,4)

** function called with positional args: (3, 4) and keyword args: {}
** return value: 5.0
** function called with positional args: (3, 4) and keyword args: {}
** return value: 5.0


5.0

In [25]:
@logger
@twice
def compute(x, y):
    "compute hypothenuse from side lengths x and y"
    print('compute....')
    return (x**2 + y**2)**.5

In [27]:
compute?

Signature: compute(x, y)
Docstring: compute hypothenuse from side lengths x and y
File:      c:\users\matth\appdata\local\temp\ipykernel_28904\3217268294.py
Type:      function

In [29]:
compute.__name__

'compute'

In [ ]:
compute(3,4)

In [ ]:
compute_super_decorated = logger(twice(twice(logger(compute))))
compute_super_decorated(3,4)

## decorator @repeat(n)
- execute twice decorated function
- return mean result

In [55]:
def repeat(n):
    def repeat_decorator(f):
        @wraps(f)
        def twice_wrapper(*args, **kwargs):
            return statistics.mean(
                f(*args, **kwargs)
                for _ in range(n)
            )
        return twice_wrapper
    return repeat_decorator

In [57]:
@repeat(10)
def compute(x, y):
    "compute hypothenuse from side lengths x and y"
    print('compute....')
    return (x**2 + y**2)**.5

compute(3,4)

compute....
compute....
compute....
compute....
compute....
compute....
compute....
compute....
compute....
compute....


5.0

In [67]:
sum2 = repeat(10)(sum)

In [69]:
sum2(range(10))

45

In [73]:
sum3 = repeat(5)(logger(sum))

In [75]:
sum3(range(10))

** function called with positional args: (range(0, 10),) and keyword args: {}
** return value: 45
** function called with positional args: (range(0, 10),) and keyword args: {}
** return value: 45
** function called with positional args: (range(0, 10),) and keyword args: {}
** return value: 45
** function called with positional args: (range(0, 10),) and keyword args: {}
** return value: 45
** function called with positional args: (range(0, 10),) and keyword args: {}
** return value: 45


45

## NB: class attribute example

In [ ]:
class A:
    cpt: int = 0

    def __init__(self):
        A.cpt +=1

In [ ]:
for _ in range(1000):
    _ = A()
A.cpt

## decorating a class

In [101]:
from dataclasses import dataclass
from typing import Literal

In [89]:
def displayable(cls):
    def display(self):
        print('str:', self)
        print('repr:', repr(self)) 
    cls.display = display
    return cls

In [95]:
@displayable
@dataclass
class Person:
    name: str

    def __str__(self):
        return self.name

In [97]:
p = Person('toto')
p.display()

str: toto
repr: Person(name='toto')


In [103]:
def displayable2(method: Literal['str','repr']):
    def displayable_decorator(cls):
        def display(self):
            match method:
                case 'str':
                    print('str:', self)
                case 'repr':
                    print('repr:', repr(self)) 
        cls.display = display
        return cls
    return displayable_decorator

In [105]:
@displayable2('str')
@dataclass
class Person:
    name: str

    def __str__(self):
        return self.name

p = Person('titi')
p.display()

str: titi


In [107]:
@displayable2('repr')
@dataclass
class Person:
    name: str

    def __str__(self):
        return self.name

p = Person('titi')
p.display()

repr: Person(name='titi')
